# Подготовка

Импортируем нужный модуль.

In [1]:
import pandas as pd

Считаем данные в датафрейм.

In [2]:
dataset = pd.read_csv('titanic.csv')

In [3]:
dataset.head(20)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


Сразу заметно, что некоторые колонки по своему смыслу дублируют другие. Немного преобразуем данные в похожих колонках, чтобы их было легче сравнить, а затем сравним такие колонки и удалим в случае их совпадения.

## Колонки sex и who

Для начала проверим, какие есть значения в колонках sex и who.

In [4]:
print(dataset.sex.value_counts(), '\n')
print(dataset.who.value_counts())

male      577
female    314
Name: sex, dtype: int64 

man      537
woman    271
child     83
Name: who, dtype: int64


Поменяем значения в колонке sex, чтобы было удобнее сравнивать её с who.

In [5]:
dataset.loc[dataset.sex == 'male', 'sex'] = 'man'
dataset.loc[dataset.sex == 'female', 'sex'] = 'woman'

Заметим, что пол одного и того же взрослого человека одинаков в обеих колонках. Единственное отличие между колонками - who содержит информацию, являлся ли пассажир ребёнком.

In [6]:
dataset[dataset.sex != dataset.who].who.value_counts()

child    83
Name: who, dtype: int64

Чтобы избежать дублирования данных, уберём колонку sex, а в колонке who добавим данные о половой принадлежности детей.

In [7]:
dataset.loc[(dataset.who == 'child') & (dataset.sex == 'man'), 'who'] = 'male_child'
dataset.loc[(dataset.who == 'child') & (dataset.sex == 'woman'), 'who'] = 'female_child'
dataset.drop(columns=['sex'], inplace=True)

Также теперь стала бесполезна колонка adult_male, так как все взрослые мужчины занесены в колонке who как male, а мальчики как male_child. Проверим, что данные о взрослых мужчинах в колонках adult_male и who совпадают.

In [8]:
print(dataset[(dataset.adult_male == True) & (dataset.who != 'man')].shape[0])
print(dataset[(dataset.adult_male == False) & (dataset.who == 'man')].shape[0])

0
0


Удалим колонку adult_male за ненадобностью.

In [9]:
dataset.drop(columns=['adult_male'], inplace=True)

## Колонки class и pclass

Проверим, что в колонках class и pclass одинаковое количество сходных по смыслу значений.

In [10]:
print(dataset['class'].value_counts(), '\n')
print(dataset['pclass'].value_counts())

Third     491
First     216
Second    184
Name: class, dtype: int64 

3    491
1    216
2    184
Name: pclass, dtype: int64


Поменяем значения в колонке class, чтобы было удобнее сравнивать её с pclass.

In [11]:
dataset.loc[dataset['class'] == 'First', 'class'] = 1
dataset.loc[dataset['class'] == 'Second', 'class'] = 2
dataset.loc[dataset['class'] == 'Third', 'class'] = 3

Видно, что колонки class и pclass полностью совпадают.

In [12]:
dataset[dataset['class'] != dataset['pclass']].shape[0]

0

Удалим одну из колонок, чтобы избежать дублирования данных.

In [13]:
dataset.drop(columns=['class'], inplace=True)

## Колонки embark_town и embarked

Проверим, какие значения есть в колонках embark_town и embarked.

In [14]:
print(dataset.embark_town.value_counts(), '\n')
print(dataset.embarked.value_counts())

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64 

S    644
C    168
Q     77
Name: embarked, dtype: int64


Поменяем значения в колонке embark_town, чтобы было удобнее сравнивать её с embarked.

In [15]:
dataset.loc[dataset.embark_town == 'Southampton', 'embark_town'] = 'S'
dataset.loc[dataset.embark_town == 'Cherbourg', 'embark_town'] = 'C'
dataset.loc[dataset.embark_town == 'Queenstown', 'embark_town'] = 'Q'

Заметим, что в колонках class и pclass есть 2 несовпадающих значения, но в этом случае в обеих колонках пропущены значения.

In [16]:
dataset[dataset.embark_town != dataset.embarked]

,survived,pclass,age,sibsp,parch,fare,embarked,who,deck,embark_town,alive,alone
61,1,1,38.0,0,0,80.0,NaN,woman,B,NaN,yes,True
829,1,1,62.0,0,0,80.0,NaN,woman,B,NaN,yes,True


Позаботимся об этом позже, а пока удалим одну из колонок, так как они дублируют друг друга.

In [17]:
dataset.drop(columns=['embark_town'], inplace=True)

## Колонки survived и alive

Проверим, какие значения есть в колонках survived и alive.

In [18]:
print(dataset.survived.value_counts(), '\n')
print(dataset.alive.value_counts())

0    549
1    342
Name: survived, dtype: int64 

no     549
yes    342
Name: alive, dtype: int64


Поменяем значения в колонке alive, чтобы было удобнее сравнивать её с survived.

In [19]:
dataset.loc[dataset.alive == 'yes', 'alive'] = 1
dataset.loc[dataset.alive == 'no', 'alive'] = 0

Колонки полностью совпадают.

In [20]:
dataset[dataset.alive != dataset.survived].shape[0]

0

Удалим одну из них, чтобы избежать дублирования данных.

In [21]:
dataset.drop(columns=['alive'], inplace=True)

## Колонка alone

Я не стал ничего делать с колонкой alone, несмотря на то что она фактически является производной от колонок sibsp и parch. Она может быть полезна при работе с этими данными, да и не дублирует эти колонки, а я избавлялся именно от колонок, в которых содержатся одинаковые сведения.

# Работа с пропущенными значениями

Взглянем на датасет ещё раз.

In [22]:
dataset.head(20)

,survived,pclass,age,sibsp,parch,fare,embarked,who,deck,alone
0,0,3,22.0,1,0,7.2500,S,man,NaN,False
1,1,1,38.0,1,0,71.2833,C,woman,C,False
2,1,3,26.0,0,0,7.9250,S,woman,NaN,True
3,1,1,35.0,1,0,53.1000,S,woman,C,False
4,0,3,35.0,0,0,8.0500,S,man,NaN,True
5,0,3,NaN,0,0,8.4583,Q,man,NaN,True
6,0,1,54.0,0,0,51.8625,S,man,E,True
7,0,3,2.0,3,1,21.0750,S,male_child,NaN,False
8,1,3,27.0,0,2,11.1333,S,woman,NaN,False
9,1,2,14.0,1,0,30.0708,C,female_child,NaN,False


"*Высота*" датасета равна 891. Колонки, в которых меньше значений, содержат пропуски.

In [23]:
dataset.shape

(891, 10)

 Пустые значения есть в колонках age, embarked, deck.

In [24]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   age       714 non-null    float64
 3   sibsp     891 non-null    int64  
 4   parch     891 non-null    int64  
 5   fare      891 non-null    float64
 6   embarked  889 non-null    object 
 7   who       891 non-null    object 
 8   deck      203 non-null    object 
 9   alone     891 non-null    bool   
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 63.6+ KB


Убедимся в этом.

In [35]:
dataset.isna().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
embarked    0
who         0
alone       0
dtype: int64

## Пропуски в embarked

S (Southampton) - самый частый город посадки.

In [25]:
dataset.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

Всего 2 пропущенных значения, поэтому можем заменить на самое популярное, качество данных от этого сильно не ухудшится.

In [26]:
dataset.embarked = dataset.embarked.fillna('S')

## Пропуски в deck

Ни одна палуба не является сильно более популярной, а пропущенных значений слишком много.

In [27]:
dataset.deck.value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64

Удалим колонку, так как она скорее всего будет мешать модели, а замена пропущенных значений может помешать ещё сильнее, так как известна информация всего по 203 значениям из 891.

In [28]:
dataset.drop(columns=['deck'], inplace=True)

## Пропуски в age

Так как пропущенных значений не слишком много (177 из 891), заменим их медианным значением возраста.

In [29]:
dataset.age = dataset.age.fillna(dataset.age.median())

## Проверка

In [33]:
dataset.head()

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
0,0,3,22.0,1,0,7.2500,S,man,False
1,1,1,38.0,1,0,71.2833,C,woman,False
2,1,3,26.0,0,0,7.9250,S,woman,True
3,1,1,35.0,1,0,53.1000,S,woman,False
4,0,3,35.0,0,0,8.0500,S,man,True


In [34]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   age       891 non-null    float64
 3   sibsp     891 non-null    int64  
 4   parch     891 non-null    int64  
 5   fare      891 non-null    float64
 6   embarked  891 non-null    object 
 7   who       891 non-null    object 
 8   alone     891 non-null    bool   
dtypes: bool(1), float64(2), int64(4), object(2)
memory usage: 56.7+ KB


In [36]:
dataset.isna().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
embarked    0
who         0
alone       0
dtype: int64

Всё в порядке: пропущенных значений нет ни в одной колонке, а в датасете содержатся только нужные колонки.

# Ответы на вопросы по исходным данным

### Пункт а

In [30]:
print('Количество мужчин:', dataset[(dataset.who == 'man') | (dataset.who == 'male_child')].shape[0])
print('Количество женщин:', dataset[(dataset.who == 'woman') | (dataset.who == 'female_child')].shape[0])

Количество мужчин: 577
Количество женщин: 314


Мужчин больше, чем женщин.

### Пункт b

Поделим количество взрослых мужчин на общее количество записей в датасете.

In [31]:
dataset[dataset.who == 'man'].shape[0] / dataset.shape[0] * 100 

60.26936026936027

Взрослых мужчин около 60,27%

### Пункт c

Посчитаем количество значений для каждого класса билета.

In [32]:
dataset.pclass.value_counts() 

3    491
1    216
2    184
Name: pclass, dtype: int64

В 3 классе был 491 пассажир, во 2-ом - 184, в 1-ом - 216.

# Выводы 

### Выводы по основному датасету

In [38]:
dataset.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


- Средний возраст пассажиров составляет приблизительно 29,4 года.
- В среднем билет стоил 32,2 доллара.
- Выжило около 38% пассажиров.
- Возраст самого молодого пассажира - 0.42 года, а самого пожилого - 80 лет. Да, слово "*пассажир*" употреблено не случайно, есть информация только об одном мальчике 0.42 лет и одном мужчине 80 лет, проверка есть в разделе *Дополнение*.
- Самый дорогой билет стоил больше 512 долларов, а кто-то не заплатил ничего. В разделе *Дополнение* проверил, что таких людей всего 15, все они мужчины, поднялись на борт в одном городе и не имели ни одного родственника на борту. Также они все совершеннолетние, так что вероятно, это кто-то из членов команды. Единственное отличие - у некоторых из них билеты разного класса.
- Самое большое количество близких родственников/супругов на борту - 8.
- Самое больше количество родителей/детей на борту - 6.

### Выводы по новому датасету

Создадим датафрейм, в котором будут только выжившие пассажиры.

In [39]:
survived_df = dataset[dataset.survived == 1]

In [40]:
survived_df.head(10)

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
1,1,1,38.0,1,0,71.2833,C,woman,False
2,1,3,26.0,0,0,7.9250,S,woman,True
3,1,1,35.0,1,0,53.1000,S,woman,False
8,1,3,27.0,0,2,11.1333,S,woman,False
9,1,2,14.0,1,0,30.0708,C,female_child,False
10,1,3,4.0,1,1,16.7000,S,female_child,False
11,1,1,58.0,0,0,26.5500,S,woman,True
15,1,2,55.0,0,0,16.0000,S,woman,True
17,1,2,28.0,0,0,13.0000,S,man,True
19,1,3,28.0,0,0,7.2250,C,woman,True


Чтобы найти процент выживших взрослых мужчин от всех выживших пассажиров, сделаем то же самое, что и с основным датафреймом.

In [41]:
survived_df[survived_df.who == 'man'].shape[0] / survived_df.shape[0] * 100 

25.730994152046783

Теперь процент взрослых мужчин опустился примерно до 25,73%.

In [42]:
survived_df.describe()

,survived,pclass,age,sibsp,parch,fare
count,342.0,342.000000,342.000000,342.000000,342.000000,342.000000
mean,1.0,1.950292,28.291433,0.473684,0.464912,48.395408
std,0.0,0.863321,13.764425,0.708688,0.771712,66.596998
min,1.0,1.000000,0.420000,0.000000,0.000000,0.000000
25%,1.0,1.000000,21.000000,0.000000,0.000000,12.475000
50%,1.0,2.000000,28.000000,0.000000,0.000000,26.000000
75%,1.0,3.000000,35.000000,1.000000,1.000000,57.000000
max,1.0,3.000000,80.000000,4.000000,5.000000,512.329200


- Уменьшилось среднее значение класса билета. Это говорит о том, что у людей с более высоким классом билета был выше шанс выжить.
- Уменьшился средний возраст, теперь он составляет примерно 28,3 года, то есть у более молодых людей был выше шанс выжить.
- Уменьшилось среднее значение родственников на борту. Те, кто были без них, выживал чаще.
- Поднялось среднее значение родителей/детей на борту. Люди, у которых были эти родственники на борту, выживали чаще.
- Выжили самый молодой и самый пожилой пассажиры.
- Все люди, заплатившие за самые дорогие билеты, выжили. Стоит заметить, что все они, кроме одного, были одни на борту (без родственников и тд). Проверка есть в разделе *Дополнение*.
- Наибольшее количество близких родственников/супругов на борту у выжившего человека - 4.
- Наибольшее количество детей/родителей на борту у выжившего человека - 5.

## Дополнение

Проверим людей, которые не заплатили за билет, чтобы убедиться, что это не ошибка в данных и таких людей не слишком много.

In [47]:
dataset[dataset.fare == 0]

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
179,0,3,36.0,0,0,0.0,S,man,True
263,0,1,40.0,0,0,0.0,S,man,True
271,1,3,25.0,0,0,0.0,S,man,True
277,0,2,28.0,0,0,0.0,S,man,True
302,0,3,19.0,0,0,0.0,S,man,True
413,0,2,28.0,0,0,0.0,S,man,True
466,0,2,28.0,0,0,0.0,S,man,True
481,0,2,28.0,0,0,0.0,S,man,True
597,0,3,49.0,0,0,0.0,S,man,True
633,0,1,28.0,0,0,0.0,S,man,True


Посмотрим на человека с самым большим возрастом на борту и проверим, что он такой один.

In [44]:
dataset[dataset.age == 0.42]

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
803,1,3,0.42,0,1,8.5167,C,male_child,False


Взглянем на самого молодого человека на борту и проверим, что он такой один.

In [45]:
dataset[dataset.age == 80]

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
630,1,1,80.0,0,0,30.0,S,man,True


Взглянем на людей, которые заплатили больше всего.

In [50]:
dataset[dataset.fare == dataset.fare.max()]

,survived,pclass,age,sibsp,parch,fare,embarked,who,alone
258,1,1,35.0,0,0,512.3292,C,woman,True
679,1,1,36.0,0,1,512.3292,C,man,False
737,1,1,35.0,0,0,512.3292,C,man,True
